In [1]:
%%html
<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at 'In [5]'.</span>

In [2]:
%%html
<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at 'In [3]'.</span>

## FDA inspection data analytics - build the dataset

## load libraries

In [3]:
file = '../data/input/raw_clinical_investigator_inspection_fc.xls'
file2 = '../data/input/raw_clinical_investigator_inspection_da.xls'
file_map = '../data/input/refined_mapping_fda_inspections_code.csv'
file_out = '../data/output/final_set_FDA.csv'

In [4]:
# Parameters
file = "data/input/raw_clinical_investigator_inspection_fc.xls"
file2 = "data/input/raw_clinical_investigator_inspection_da.xls"
file_map = "data/input/refined_mapping_fda_inspections_code.csv"
file_out = "data/output/final_set_FDA.csv"


In [5]:
import numpy as np
import os as os
import pandas as pd
import xlrd 


## load and clean fda inspections data

In [6]:
xl = pd.ExcelFile(file)

df_fc = xl.parse('Sheet1')


xl = pd.ExcelFile(file2)

df_da = xl.parse('Sheet1')

In [7]:
print(df_da.shape)
print (df_fc.shape)

(2500, 12)
(1774, 12)


In [8]:
# append the two df
df_insp = df_da.append(df_fc, ignore_index = True)

In [9]:
print(df_insp.shape)

(4274, 12)


In [10]:
df_insp

,ID No.,Name,Location,Address,City,State,Cntry,Zip,Insp. Date,Insp. Type,Insp. Class.,Insp. Def. Codes
0,44,"Chey, William Y MD",Univ Rochester,601 Elmwood Ave Box 646,Rochester,NY,USA,14642,08/18/00,DA,VAI,05
1,44,"Chey, William Y MD",Univ Rochester,601 Elmwood Ave Box 646,Rochester,NY,USA,14642,09/03/86,DA,VAI2,03
2,66,"Davidov, Michael E MD",-,311 Park Ave,Falls Church,VA,USA,22046,01/13/93,DA,NAI,00
3,99,"Ciancio, Sebastian G DDS",State Univ @ New York Of Buffalo,3435 Main St/250 Squire Hall,Buffalo,NY,USA,14214,12/07/15,DA,NAI,00
4,99,"Ciancio, Sebastian G DDS",State Univ @ New York Of Buffalo,3435 Main St/250 Squire Hall,Buffalo,NY,USA,14214,02/03/97,DA,VAI,03
...,...,...,...,...,...,...,...,...,...,...,...,...
4269,719450,"Hassan, Kazi MD",Florida Medical Pain Management,6333 54Th Avenue North,St Petersburg,FL,USA,33709,03/05/19,FC,NAI,00
4270,719923,"Yi, Sung H MD",Advanced Research Center Inc,"1020 S Anaheim Blvd, Suite 316",Anaheim,CA,USA,92805,06/18/19,FC,NAI,00
4271,720510,"Huang, May",NaN,1221 Madison Street Suite 1410,Seattle,WA,USA,98104,11/15/18,FC,VAI,05
4272,721608,"Cho, Austina B MD",Advanced Reserch Center Inc,"1020 S Anaheim Blvd, Suite 316",Anaheim,CA,USA,92805,06/18/19,FC,NAI,00


In [11]:
# rename the 'Insp. Def. codes' to 'insp_codes' and 
df_insp.rename(columns={'Insp. Def. Codes':'insp_codes'}, inplace=True)
df_insp.rename(columns={'Insp. Class.':'insp_class'}, inplace=True)

In [12]:
# convert code '00' and '0' (=no finding noted) into '22' to avoid issues mixing up with NaN
df_insp['insp_codes'].replace(['00'], '22', inplace=True)
df_insp['insp_codes'].replace(['0'], '22', inplace=True)

In [13]:
# convert empty/blank cells in 'insp_codes'(=with no inspection data) into NaN
df_insp = df_insp.replace(r'^\s+$', np.nan, regex=True)

In [14]:
# drop washout inspections
df_insp1 = df_insp[df_insp.insp_class != 'WASH']
print(df_insp1.shape)

(4160, 12)


In [15]:
# drop canceled inspections
insp_data = df_insp1[df_insp1.insp_class != 'CANC']
print(insp_data.shape)

(3972, 12)


In [16]:
# look for inspections with no action required 'NAI' but w/o code assigned

insp_data_NULL = insp_data[insp_data['insp_codes'].isnull()]
print(insp_data_NULL.shape)

(937, 12)


In [17]:
insp_data_NAI_and_null = insp_data_NULL[insp_data_NULL['insp_class'] == 'NAI']
print(insp_data_NAI_and_null.shape)

(571, 12)


In [18]:
# there are inspections with insp_class = No Action Indicated (NAI) w/o inspection code
# assign them 'insp_code' = 22

insp_data.loc[(insp_data['insp_class'] == 'NAI'), 'insp_codes'] = '22'
print(insp_data.shape)

(3972, 12)


/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [19]:
# check numbers add up

print((insp_data[insp_data['insp_codes'].isnull()]).shape)

# 961 rows with NaN = 639 insp_class = 'NAI' AND insp_code= '22' + 322 insp_code = NaN
# ---> OK

(366, 12)


In [20]:
# look for rows with typos
value_list = [',', ',,',',,,']

In [21]:
insp_data[insp_data.insp_codes.isin(value_list)]

,ID No.,Name,Location,Address,City,State,Cntry,Zip,Insp. Date,Insp. Type,insp_class,insp_codes
520,14835,"Shalita, Alan R MD",Suny Hlth Sci Ctr ...,450 Clarkson Ave Box 46,Brooklyn,NY,USA,11203,08/08/83,DA,VAI3F,","
521,14835,"Shalita, Alan R MD",Suny Hlth Sci Ctr ...,450 Clarkson Ave Box 46,Brooklyn,NY,USA,11203,08/08/83,DA,VAI3R,","
1503,56699,"Scharf, Martin B PHD",Ctr Res Sleep Disorders/Tristate Sleep Disorde...,1275 E Kemper Rd,Cincinnati,OH,USA,45246,02/06/01,DA,OAIW,","
2297,83034,"Trapp, George A MD",Va Med Ctr,4500 S Lancaster Rd,Dallas,TX,USA,75216,10/16/80,DA,VAI2,","
2783,99809,"Moskowitz, George MD",-,101 Rutledge St,Charlestown,SC,USA,00000,05/29/81,FC,OAIC,","


In [22]:
# replace typo with '23' (=no data available)
insp_data['insp_codes'].replace([','], '23', inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:6786: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [23]:
# replace typo (,00) - replace by 22
insp_data['insp_codes'].replace([',00'], '22', inplace=True)

In [24]:
insp_data

,ID No.,Name,Location,Address,City,State,Cntry,Zip,Insp. Date,Insp. Type,insp_class,insp_codes
0,44,"Chey, William Y MD",Univ Rochester,601 Elmwood Ave Box 646,Rochester,NY,USA,14642,08/18/00,DA,VAI,05
1,44,"Chey, William Y MD",Univ Rochester,601 Elmwood Ave Box 646,Rochester,NY,USA,14642,09/03/86,DA,VAI2,03
2,66,"Davidov, Michael E MD",-,311 Park Ave,Falls Church,VA,USA,22046,01/13/93,DA,NAI,22
3,99,"Ciancio, Sebastian G DDS",State Univ @ New York Of Buffalo,3435 Main St/250 Squire Hall,Buffalo,NY,USA,14214,12/07/15,DA,NAI,22
4,99,"Ciancio, Sebastian G DDS",State Univ @ New York Of Buffalo,3435 Main St/250 Squire Hall,Buffalo,NY,USA,14214,02/03/97,DA,VAI,03
...,...,...,...,...,...,...,...,...,...,...,...,...
4269,719450,"Hassan, Kazi MD",Florida Medical Pain Management,6333 54Th Avenue North,St Petersburg,FL,USA,33709,03/05/19,FC,NAI,22
4270,719923,"Yi, Sung H MD",Advanced Research Center Inc,"1020 S Anaheim Blvd, Suite 316",Anaheim,CA,USA,92805,06/18/19,FC,NAI,22
4271,720510,"Huang, May",NaN,1221 Madison Street Suite 1410,Seattle,WA,USA,98104,11/15/18,FC,VAI,05
4272,721608,"Cho, Austina B MD",Advanced Reserch Center Inc,"1020 S Anaheim Blvd, Suite 316",Anaheim,CA,USA,92805,06/18/19,FC,NAI,22


In [25]:
# explose insp_codes column (requires pandas >0.25)
insp_data_interim = insp_data.assign(insp_codes=df_insp.insp_codes.str.split(',')).explode('insp_codes')

In [26]:
# check
insp_data_interim

,ID No.,Name,Location,Address,City,State,Cntry,Zip,Insp. Date,Insp. Type,insp_class,insp_codes
0,44,"Chey, William Y MD",Univ Rochester,601 Elmwood Ave Box 646,Rochester,NY,USA,14642,08/18/00,DA,VAI,05
1,44,"Chey, William Y MD",Univ Rochester,601 Elmwood Ave Box 646,Rochester,NY,USA,14642,09/03/86,DA,VAI2,03
2,66,"Davidov, Michael E MD",-,311 Park Ave,Falls Church,VA,USA,22046,01/13/93,DA,NAI,22
3,99,"Ciancio, Sebastian G DDS",State Univ @ New York Of Buffalo,3435 Main St/250 Squire Hall,Buffalo,NY,USA,14214,12/07/15,DA,NAI,22
4,99,"Ciancio, Sebastian G DDS",State Univ @ New York Of Buffalo,3435 Main St/250 Squire Hall,Buffalo,NY,USA,14214,02/03/97,DA,VAI,03
...,...,...,...,...,...,...,...,...,...,...,...,...
4269,719450,"Hassan, Kazi MD",Florida Medical Pain Management,6333 54Th Avenue North,St Petersburg,FL,USA,33709,03/05/19,FC,NAI,22
4270,719923,"Yi, Sung H MD",Advanced Research Center Inc,"1020 S Anaheim Blvd, Suite 316",Anaheim,CA,USA,92805,06/18/19,FC,NAI,22
4271,720510,"Huang, May",NaN,1221 Madison Street Suite 1410,Seattle,WA,USA,98104,11/15/18,FC,VAI,05
4272,721608,"Cho, Austina B MD",Advanced Reserch Center Inc,"1020 S Anaheim Blvd, Suite 316",Anaheim,CA,USA,92805,06/18/19,FC,NAI,22


In [27]:
# convert insp_code into numeric data for the join
insp_data_interim['insp_codes'] = pd.to_numeric(insp_data_interim['insp_codes'])

In [28]:
# Verify values in insp_codes
print(insp_data_interim.insp_codes.unique())

[ 5.  3. 22. 15.  6. nan  0.  4.  7. 16. 12. 14. 18.  1.  2.  8. 10. 13.
 11.  9. 17. 20. 19.]


In [29]:
# check 
print(insp_data_interim[insp_data_interim['insp_codes'].isnull()])

      ID No.                              Name  \
7        113   Vlachakis,   Nicholas   D    MD   
8        113   Vlachakis,   Nicholas   D    MD   
9        146      Homesley,   Howard   D    MD   
14       271          Lipton,   Allan       MD   
31       748    Pendergrass,   Kelly   B    MD   
...      ...                               ...   
4171  642028                Chisti,   Mohammad   
4178  648641                   Driver,   Brian   
4195  661444  Castellon,   Celestino   P    MD   
4201  666448                  Racke,   Michael   
4254  697186                Meltzer,   Herbert   

                                               Location  \
7                           Los Angeles Cty/Usc Med Ctr   
8                           Los Angeles Cty/Usc Med Ctr   
9                  Gynecology Oncology Middle Tennessee   
14                             Milton S Hershey Med Ctr   
31                             Kansas City Internal Med   
...                                          

In [30]:
#find duplicates

duplicateRowsDF = insp_data_interim[insp_data_interim.duplicated()]

print(duplicateRowsDF)


      ID No.                          Name  \
37       839      Arnold,   John   D    MD   
162     4682   Bodey,   Gerald   P  SR  MD   
162     4682   Bodey,   Gerald   P  SR  MD   
360    10639     Gentry,   Layne   O    MD   
360    10639     Gentry,   Layne   O    MD   
...      ...                           ...   
3874  499117  Montero,   Alberto   J    MD   
4041  542883      Renfroe,   Ben   J    MD   
4208  669694   Huffman,   Cynthia       MD   
4240  683889             Lang,   Jason   E   
4241  684207                Amin,   Faisal   

                                Location                              Address  \
37                                     -                       5104 E 24Th St   
162        Univ Texas Md Anderson Ca Ctr                   1515 Holcombe Blvd   
162        Univ Texas Md Anderson Ca Ctr                   1515 Holcombe Blvd   
360              St Lukes Episcopal Hosp                     6720 Bertner Ave   
360              St Lukes Episcopal Hosp  

In [31]:
# drop duplicates
insp_data_final = insp_data_interim.drop_duplicates(keep='first')

In [32]:
# confirm # of rows with NaN
print((insp_data_final[insp_data_final['insp_codes'].isnull()]).shape)

(1024, 12)


## load mapping data 

In [33]:
# load csv file - created and refined based on:
# https://www.fda.gov/drugs/enforcement-activities-fda/clinical-investigator-inspection-list-ciil-database-codes)
mapping = pd.read_csv (file_map, encoding = 'cp1250',low_memory=False)

In [34]:
# check
mapping

,code,deficiency,cfr_reference,mapped_clinical_impact_factor
0,1,Records availability,21 CFR 312.62,data_integrity
1,2,Failure to obtain and/or document subject consent,"21 CFR 312.60, 50.20, 50.27",consent
2,3,Inadequate informed consent form,21 CFR 50.25,consent
3,4,Inadequate drug accountability,"21 CFR 312.60, 312.62",primary_endpoints_and_safety
4,5,Failure to follow investigational plan,21 CFR 312.60,primary_endpoints_and_safety
5,6,Inadequate and inaccurate records,21 CFR 312.62,primary_endpoints
6,7,Unapproved concomitant therapy,21 CFR 312.60,primary_endpoints_and_safety
7,8,Inappropriate payment to volunteers,21 CFR 50.20,consent
8,9,Unapproved use of drug before IND submission,21 CFR 312.40(d),safety
9,10,Inappropriate delegation of authority,"21 CFR 312.7, 312.61",primary_endpoints_and_safety


In [35]:
# rename column to insp_codes to allow mapping with FDA dataset
mapping.rename(columns={'code':'insp_codes'}, inplace=True)

In [36]:
# insp_codes to numeric value
mapping['insp_codes'] = pd.to_numeric(mapping['insp_codes'])

## join and clean consolidated data set ('fda_inspections')

In [37]:
# do the join
finalset = pd.merge (insp_data_final,mapping, on='insp_codes', how='left')

In [38]:
# check
print(finalset.shape)
finalset.info()

(7472, 15)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 7472 entries, 0 to 7471
Data columns (total 15 columns):
ID No.                           7472 non-null int64
Name                             7472 non-null object
Location                         7469 non-null object
Address                          7472 non-null object
City                             7472 non-null object
State                            7472 non-null object
Cntry                            7472 non-null object
Zip                              7472 non-null object
Insp. Date                       7455 non-null object
Insp. Type                       7472 non-null object
insp_class                       7472 non-null object
insp_codes                       6448 non-null float64
deficiency                       6421 non-null object
cfr_reference                    5847 non-null object
mapped_clinical_impact_factor    6421 non-null object
dtypes: float64(1), int64(1), object(13)
memory usage: 934.0+ KB


## final cleaning

In [39]:
print(finalset.insp_codes.unique())

[ 5.  3. 22. 15.  6. nan  0.  4.  7. 16. 12. 14. 18.  1.  2.  8. 10. 13.
 11.  9. 17. 20. 19.]


In [40]:
print(finalset[finalset['insp_codes'].isnull()])

      ID No.                              Name  \
11       113   Vlachakis,   Nicholas   D    MD   
12       113   Vlachakis,   Nicholas   D    MD   
13       146      Homesley,   Howard   D    MD   
26       271          Lipton,   Allan       MD   
57       748    Pendergrass,   Kelly   B    MD   
...      ...                               ...   
7342  642028                Chisti,   Mohammad   
7351  648641                   Driver,   Brian   
7372  661444  Castellon,   Celestino   P    MD   
7379  666448                  Racke,   Michael   
7449  697186                Meltzer,   Herbert   

                                               Location  \
11                          Los Angeles Cty/Usc Med Ctr   
12                          Los Angeles Cty/Usc Med Ctr   
13                 Gynecology Oncology Middle Tennessee   
26                             Milton S Hershey Med Ctr   
57                             Kansas City Internal Med   
...                                          

In [41]:
# Add insp_code '23' --> "no data reported"

finalset['insp_codes'].fillna(23, inplace=True)

In [42]:
# confirm no NaN
print((finalset[finalset['insp_codes'].isnull()]).shape)

(0, 15)


In [43]:
# Export to csv the final set
finalset.to_csv(file_out)